In [ ]:
# -*- coding: utf-8 -*-
"""
Notebook: Todos os Exercícios do Tutorial
Tutorial 2: Ciência de Dados Aplicada à Engenharia Agrícola – UFV

Objetivo: Reunir todos os exercícios do curso com soluções completas e comentadas.
"""

# %% Importar bibliotecas necessárias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.stats import ttest_ind
import warnings
warnings.filterwarnings("ignore")

print("✅ Bibliotecas carregadas. Vamos começar!")

In [ ]:
# %% [markdown]
# **Exercício 1: Manipulação de arrays NumPy – Umidade do solo**
#
# Objetivo: Criar um array 5x7 com dados de umidade do solo (8–25%) e calcular:
# a) Média diária
# b) Profundidade com maior variação
# c) Dias com média geral abaixo de 12%
#
# Aplicação: análise de perfis de umidade em campo

# Dados simulados: 5 profundidades x 7 dias
np.random.seed(42)
umidade = np.random.uniform(8, 25, size=(5, 7))

# a) Média diária (por coluna)
media_diaria = umidade.mean(axis=0)

# b) Profundidade com maior variação (por linha)
var_profundidade = umidade.var(axis=1)
prof_maior_var = np.argmax(var_profundidade)  # índice da profundidade

# c) Dias com média geral < 12%
media_geral_dia = umidade.mean(axis=0)
dias_criticos = np.where(media_geral_dia < 12)[0]

print("🔍 Resultados - Exercício 1")
print(f"Média diária: {media_diaria}")
print(f"Profundidade com maior variação: {prof_maior_var} (índice)")
print(f"Dias com média < 12%: {dias_criticos}")

In [ ]:
# %% [markdown]
# **Exercício 2: Filtros e máscaras com NumPy – Temperatura do ar**
#
# Objetivo: Dado um array 24x7 com temperaturas horárias:
# a) Encontre os dias com mais de 5h acima de 32°C
# b) Calcule a amplitude térmica diária
# c) Identifique o horário médio de pico de temperatura
#
# Aplicação: avaliação de estresse térmico em culturas

# Dados simulados
np.random.seed(43)
temp_horario = np.random.normal(28, 4, size=(24, 7))  # 24h x 7 dias

# a) Dias com >5h acima de 32°C
acima_32 = temp_horario > 32
horas_por_dia = acima_32.sum(axis=0)
dias_criticos = np.where(horas_por_dia > 5)[0]

# b) Amplitude térmica diária
amp_diaria = temp_horario.max(axis=0) - temp_horario.min(axis=0)

# c) Horário médio de pico
horario_pico = np.argmax(temp_horario, axis=0)
hora_media_pico = np.mean(horario_pico)

print("🔍 Resultados - Exercício 2")
print(f"Dias com >5h >32°C: {dias_criticos}")
print(f"Amplitude térmica média: {amp_diaria.mean():.2f} °C")
print(f"Horário médio de pico: {hora_media_pico:.1f} h")

In [ ]:
# %% [markdown]
# **Exercício 3: Leitura e filtragem com Pandas – Precipitação**
#
# Objetivo: Ler dados de precipitação e:
# a) Filtrar dias com chuva > 15 mm
# b) Selecionar apenas a estação "Campo A"
# c) Criar média móvel de 3 dias
#
# Aplicação: análise de risco de encharcamento

# Dados simulados
np.random.seed(44)
datas = pd.date_range("2024-01-01", periods=100)
df_chuva = pd.DataFrame({
    'data': datas,
    'estacao': np.random.choice(['Campo A', 'Campo B', 'Laboratório'], 100),
    'chuva_mm': np.random.exponential(6, 100).clip(0, 50)
})

# a) Chuva > 15 mm
chuva_alta = df_chuva[df_chuva['chuva_mm'] > 15]

# b) Estação "Campo A"
campo_a = df_chuva[df_chuva['estacao'] == 'Campo A']

# c) Média móvel de 3 dias
df_chuva['media_movel_3d'] = df_chuva['chuva_mm'].rolling(3, min_periods=1).mean()

print("🔍 Resultados - Exercício 3")
print(f"Dias com chuva >15 mm: {len(chuva_alta)}")
print(f"Dados da estação Campo A: {len(campo_a)} linhas")
print("Primeiras 5 médias móveis:")
print(df_chuva[['chuva_mm', 'media_movel_3d']].head())

In [ ]:
# %% [markdown]
# **Exercício 4: Limpeza de dados com Pandas – Vazão de irrigação**
#
# Objetivo: Dado um DataFrame com 10% de NaN:
# a) Preencher com média móvel de 3 pontos
# b) Remover linhas com pressão < 0.5 bar
# c) Criar coluna com total diário de água
#
# Aplicação: tratamento de falhas em sensores de irrigação

# Dados simulados
np.random.seed(45)
vazao = np.random.normal(2.0, 0.3, 50)
vazao[np.random.choice(50, 5, replace=False)] = np.nan  # 10% NaN
df_irr = pd.DataFrame({
    'data': pd.date_range("2024-01-01", periods=50, freq='H'),
    'pressão_bar': np.random.normal(2.0, 0.5, 50).clip(0.2, 3.0),
    'vazao_lh': vazao
})

# a) Preencher NaN com média móvel
df_irr['vazao_lh'] = df_irr['vazao_lh'].fillna(df_irr['vazao_lh'].rolling(3, min_periods=1).mean())

# b) Remover pressão < 0.5 bar
df_irr = df_irr[df_irr['pressão_bar'] >= 0.5]

# c) Total diário de água (m³/dia)
df_irr['data_dia'] = df_irr['data'].dt.date
total_diario = df_irr.groupby('data_dia')['vazao_lh'].sum() * 3.6 / 1000  # L/h → m³/dia

print("🔍 Resultados - Exercício 4")
print(f"Total diário médio: {total_diario.mean():.2f} m³/dia")
print(f"Dados remanescentes: {len(df_irr)} linhas")

In [ ]:
# %% [markdown]
# **Exercício 5: Agregação com Pandas – Consumo de tratores**
#
# Objetivo: Analisar dados de tratores:
# a) Calcule consumo médio por modelo
# b) Combine com tabela de manutenção
# c) Crie pivot table por modelo e turno
#
# Aplicação: análise de eficiência operacional

# Dados principais
modelos = ['Valtra 120', 'Massey 285', 'John Deere 5075']
df_trator = pd.DataFrame({
    'modelo': np.random.choice(modelos, 100),
    'turno': np.random.choice(['manhã', 'tarde', 'noite'], 100),
    'consumo_lh': np.random.normal(9, 1, 100)
})

# Tabela de manutenção (simulada)
df_mant = pd.DataFrame({
    'modelo': modelos,
    'ult_manutencao': ['2024-01-15', '2024-02-10', '2024-01-30']
})

# a) Consumo médio por modelo
consumo_medio = df_trator.groupby('modelo')['consumo_lh'].mean()

# b) Merge com manutenção
df_completo = pd.merge(df_trator, df_mant, on='modelo', how='left')

# c) Pivot table
pivot = pd.pivot_table(df_trator, values='consumo_lh', index='modelo', columns='turno', aggfunc='mean')

print("🔍 Resultados - Exercício 5")
print("Consumo médio por modelo:")
print(consumo_medio.round(2))
print("\nPivot table (consumo por turno):")
print(pivot.round(2))

In [ ]:
# %% [markdown]
# **Exercício 6: Visualização – Umidade do solo em profundidades**
#
# Objetivo: Plote umidade em 4 profundidades ao longo do tempo com:
# a) Linhas com cores diferentes
# b) Título e rótulos
# c) Salvar como PDF
#
# Aplicação: monitoramento de sensores de umidade

# Dados simulados
tempo = np.arange(10)
profundidades = ['0-10cm', '10-20cm', '20-30cm', '30-40cm']
dados_umidade = np.array([20-2*t + np.random.normal(0,1,10) for t in range(4)])

plt.figure(figsize=(10, 6))
for i, prof in enumerate(profundidades):
    plt.plot(tempo, dados_umidade[i], label=prof, linewidth=2)

plt.title('Evolução da Umidade do Solo em Diferentes Profundidades', fontsize=14)
plt.xlabel('Tempo (dias)')
plt.ylabel('Umidade (%)')
plt.legend(title='Profundidade')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('umidade_solo.pdf', dpi=300, bbox_inches='tight')  # Salva como PDF
plt.show()

print("✅ Gráfico salvo como 'umidade_solo.pdf'")

In [ ]:
# %% [markdown]
# **Exercício 6: Visualização – Umidade do solo em profundidades**
#
# Objetivo: Plote umidade em 4 profundidades ao longo do tempo com:
# a) Linhas com cores diferentes
# b) Título e rótulos
# c) Salvar como PDF
#
# Aplicação: monitoramento de sensores de umidade

# Dados simulados
tempo = np.arange(10)
profundidades = ['0-10cm', '10-20cm', '20-30cm', '30-40cm']
dados_umidade = np.array([20-2*t + np.random.normal(0,1,10) for t in range(4)])

plt.figure(figsize=(10, 6))
for i, prof in enumerate(profundidades):
    plt.plot(tempo, dados_umidade[i], label=prof, linewidth=2)

plt.title('Evolução da Umidade do Solo em Diferentes Profundidades', fontsize=14)
plt.xlabel('Tempo (dias)')
plt.ylabel('Umidade (%)')
plt.legend(title='Profundidade')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('umidade_solo.pdf', dpi=300, bbox_inches='tight')  # Salva como PDF
plt.show()

print("✅ Gráfico salvo como 'umidade_solo.pdf'")

In [ ]:
# %% [markdown]
# **Exercício 7: Boxplot e histograma – Diâmetro de frutos**
#
# Objetivo: Compare 3 tratamentos de irrigação com:
# a) Boxplot
# b) Histograma sobreposto
# c) Barras com erro padrão
#
# Aplicação: classificação de frutos por tamanho

# Dados simulados
trat_a = np.random.normal(5.0, 0.8, 30)  # mm
trat_b = np.random.normal(6.2, 0.9, 30)
trat_c = np.random.normal(5.8, 0.7, 30)

# a) Boxplot
plt.figure(figsize=(12, 4))
plt.subplot(1, 3, 1)
plt.boxplot([trat_a, trat_b, trat_c], labels=['A', 'B', 'C'])
plt.title('Boxplot')

# b) Histograma sobreposto
plt.subplot(1, 3, 2)
plt.hist(trat_a, alpha=0.6, label='A', bins=8)
plt.hist(trat_b, alpha=0.6, label='B', bins=8)
plt.hist(trat_c, alpha=0.6, label='C', bins=8)
plt.title('Histograma')
plt.legend()

# c) Barras com erro
medias = [trat_a.mean(), trat_b.mean(), trat_c.mean()]
erros = [trat_a.std(), trat_b.std(), trat_c.std()]
plt.subplot(1, 3, 3)
plt.bar(['A','B','C'], medias, yerr=erros, capsize=5, color=['C0','C1','C2'])
plt.title('Média com erro')
plt.tight_layout()
plt.show()

In [ ]:
# %% [markdown]
# **Exercício 8: Análise exploratória – Evapotranspiração**
#
# Objetivo: Analise ETc de 3 culturas:
# a) Plote evolução temporal
# b) Calcule correlação com radiação
# c) Gere boxplot comparativo
#
# Aplicação: planejamento de irrigação

# %% Dados simulados
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

datas = pd.date_range("2024-01-01", periods=90)
culturas = np.random.choice(['milho', 'feijao', 'soja'], 90)

# Dicionário de médias por cultura
medias_et = {'milho': 3.2, 'feijao': 2.1, 'soja': 2.8}

# Gerar ETc com base na cultura
etcs = [np.random.normal(medias_et[cultura], 0.3) for cultura in culturas]

df_et = pd.DataFrame({
    'data': datas,
    'cultura': culturas,
    'ETc': etcs,
    'radiacao': np.random.normal(25, 5, 90)  # dados de radiação
})

# %% a) Evolução temporal
df_pivot = df_et.pivot(index='data', columns='cultura', values='ETc')
df_pivot.plot(title='Evapotranspiração das culturas', xlabel='Data', ylabel='ETc (mm/dia)')
plt.show()

# %% b) Correlação entre ETc e radiação
corr = df_et['ETc'].corr(df_et['radiacao'])
print(f"Correlação ETc vs. radiação: {corr:.3f}")

# %% c) Boxplot comparativo
df_et.boxplot(column='ETc', by='cultura', grid=False)
plt.suptitle('')
plt.title('Distribuição de ETc por cultura')
plt.show()

In [ ]:
# %% [markdown]
# **Exercício 9: Teste estatístico – Rendimento de grãos**
#
# Objetivo: Compare dois sistemas de plantio:
# a) Teste t independente
# b) Calcule tamanho do efeito (Cohen's d)
# c) Interprete p-valor
#
# Aplicação: validação de tratamentos experimentais

# Dados simulados
sistema_a = np.random.normal(3.2, 0.4, 30)  # Plantio direto
sistema_b = np.random.normal(3.0, 0.5, 30)  # Convencional

# a) Teste t
t, p = ttest_ind(sistema_a, sistema_b)

# b) Cohen's d
mean_diff = sistema_a.mean() - sistema_b.mean()
pooled_std = np.sqrt(((len(sistema_a)-1)*sistema_a.std()**2 + (len(sistema_b)-1)*sistema_b.std()**2) / (len(sistema_a)+len(sistema_b)-2))
cohens_d = mean_diff / pooled_std

print("🔍 Resultados - Exercício 9")
print(f"Teste t: t={t:.2f}, p={p:.4f}")
print(f"Cohen's d: {cohens_d:.3f}")
print("Interpretação:", "Diferença significativa" if p < 0.05 else "Sem diferença significativa")
if cohens_d > 0.8:
    print("→ Tamanho do efeito grande (relevância prática)")

In [ ]:
# %% [markdown]
# **Exercício 10: Ajuste de curva – Secagem de grãos**
#
# Objetivo: Ajuste modelo logístico à curva de crescimento
# a) Defina a função logística
# b) Ajuste com curve_fit
# c) Plote dados e curva
#
# Aplicação: modelagem de crescimento de mudas

# Dados simulados
tempo = np.linspace(0, 10, 11)
altura = 40 / (1 + np.exp(-1.2*(tempo-5))) + np.random.normal(0, 1, len(tempo))

# a) Modelo logístico
def modelo_logistico(t, K, r, t0):
    return K / (1 + np.exp(-r * (t - t0)))

# b) Ajuste
params, _ = curve_fit(modelo_logistico, tempo, altura)

# c) Plot
plt.figure(figsize=(8, 5))
plt.scatter(tempo, altura, label='Dados', color='red')
plt.plot(tempo, modelo_logistico(tempo, *params), 'b-', label='Ajuste')
plt.xlabel('Tempo (dias)')
plt.ylabel('Altura (cm)')
plt.title('Ajuste de Modelo Logístico à Curva de Crescimento')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print("Parâmetros ajustados:")
print(f"K (capacidade): {params[0]:.2f} cm")
print(f"r (taxa): {params[1]:.2f} dia⁻¹")
print(f"t0 (ponto de inflexão): {params[2]:.2f} dias")

In [ ]:
# -*- coding: utf-8 -*-
"""
Notebook: Exercício 11 – Análise de sistema de irrigação por gotejamento
Curso: Ciência de Dados para Engenharia Agrícola – UFV
Correção: Geração correta de dados por grupo para evitar vieses
"""

# %% Importar bibliotecas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# %% Dados simulados por tratamento (corrigido)
np.random.seed(42)
n_por_tratamento = 30  # 30 linhas por tratamento
tratamentos = ['Baixa pressão', 'Média pressão', 'Alta pressão']

# Listas para armazenar os dados
pressao_list = []
vazao_list = []
uniformidade_list = []
tratamento_list = []

# Parâmetros por tratamento
parametros = {
    'Baixa pressão': {'pressão': 1.0, 'vazao': 1.5, 'uniformidade': 75},
    'Média pressão': {'pressão': 2.0, 'vazao': 2.0, 'uniformidade': 85},
    'Alta pressão': {'pressão': 3.0, 'vazao': 2.3, 'uniformidade': 80}
}

# Gerar dados para cada tratamento
for trat in tratamentos:
    n = n_por_tratamento
    p = parametros[trat]
    
    pressao = np.random.normal(p['pressão'], 0.3, n)
    vazao = np.random.normal(p['vazao'], 0.2, n)
    uniformidade = np.random.normal(p['uniformidade'], 5, n).clip(60, 95)
    
    # Estender listas
    tratamento_list.extend([trat] * n)
    pressao_list.extend(pressao)
    vazao_list.extend(vazao)
    uniformidade_list.extend(uniformidade)

# Criar DataFrame
df = pd.DataFrame({
    'tratamento': tratamento_list,
    'pressão_bar': pressao_list,
    'vazao_lh': vazao_list,
    'uniformidade_%': uniformidade_list
})

# Adicionar data (simulada)
df['data'] = pd.date_range("2024-01-01", periods=len(df), freq='D')

# Embaralhar o DataFrame para simular ordem real de coleta
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# %% 1. Limpeza de dados
print("🔍 Etapa 1: Limpeza de dados")
print(f"Total de linhas: {len(df)}")
print(f"Valores faltantes:\n{df.isna().sum()}")

# %% 2. Análise estatística por tratamento
print("\n📊 Etapa 2: Análise por tratamento")
analise = df.groupby('tratamento').agg({
    'pressão_bar': ['mean', 'std'],
    'vazao_lh': ['mean', 'std'],
    'uniformidade_%': ['mean', 'std']
}).round(2)
print(analise)

# %% 3. Visualização
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Boxplot de uniformidade
df.boxplot(column='uniformidade_%', by='tratamento', ax=ax1, grid=False)
ax1.set_title('Uniformidade por tratamento')
ax1.set_ylabel('Uniformidade (%)')
ax1.set_xlabel('')

# Scatter: vazão vs pressão
cores = {'Baixa pressão': 'C0', 'Média pressão': 'C1', 'Alta pressão': 'C2'}
for trat in tratamentos:
    subset = df[df['tratamento'] == trat]
    ax2.scatter(subset['pressão_bar'], subset['vazao_lh'], 
                label=f'{trat} (n={n_por_tratamento})', 
                alpha=0.7, color=cores[trat], edgecolor='k', s=60)

ax2.set_xlabel('Pressão (bar)')
ax2.set_ylabel('Vazão (L/h)')
ax2.set_title('Relação Pressão vs Vazão')
ax2.legend(title='Tratamento', fontsize=9)
ax2.grid(True, alpha=0.3)

plt.suptitle('Análise de Sistema de Irrigação por Gotejamento', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

# %% 4. Interpretação
print("\n📝 Conclusão")
print("✅ O tratamento com 'Média pressão' apresentou a melhor uniformidade média (84.7%)")
print("⚠️ A alta pressão aumentou a vazão, mas não melhorou a uniformidade e pode causar desgaste")
print("💡 Recomenda-se operar com pressão média (2.0 bar) para equilibrar eficiência e durabilidade")

In [ ]:
# -*- coding: utf-8 -*-
"""
Notebook: Exercício 12 – Modelagem preditiva da perda de calda
"""

# %% Importar
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats

# %% Dados simulados
np.random.seed(43)
vento_ms = np.random.uniform(1, 8, 50)
perda_calda = 5 + 3 * vento_ms + 0.8 * vento_ms**2 + np.random.normal(0, 2, 50)
perda_calda = np.clip(perda_calda, 5, 100)  # limite físico

df = pd.DataFrame({'vento_ms': vento_ms, 'perda_calda_%': perda_calda})

# %% Regressão linear
slope, intercept, r_value, p_value, std_err = stats.linregress(vento_ms, perda_calda)
r2 = r_value**2

# Modelo preditivo
def prever_perda(v):
    return slope * v + intercept

# %% Plot
plt.figure(figsize=(9, 6))
plt.scatter(vento_ms, perda_calda, alpha=0.7, label='Dados experimentais')
x_fit = np.linspace(1, 8, 100)
y_fit = prever_perda(x_fit)
plt.plot(x_fit, y_fit, 'r-', label=f'Ajuste: y = {slope:.2f}x + {intercept:.2f}\nR² = {r2:.3f}')

plt.xlabel('Velocidade do vento (m/s)')
plt.ylabel('Perda de calda (%)')
plt.title('Modelagem da Perda de Calda em Função do Vento')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# %% Interpretação
print("🔍 Resultados")
print(f"R² = {r2:.3f} → O modelo explica {r2*100:.1f}% da variação.")
print(f"Para vento > 5 m/s, a perda média é {prever_perda(5):.1f}% (limite operacional).")
print("💡 Recomendação: evitar pulverização quando vento > 5 m/s.")

In [ ]:
# -*- coding: utf-8 -*-
"""
Notebook: Exercício 13 – Análise de dados climáticos horários
"""

# %% Importar
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# %% Dados horários simulados (7 dias)
hours = pd.date_range("2024-06-01", periods=168, freq='H')  # 7 dias
radiacao = np.clip(
    800 * np.sin(2 * np.pi * (hours.hour - 6) / 24) + np.random.normal(0, 50, 168),
    0, 1100
)
temp_c = 25 + 10 * np.sin(2 * np.pi * (hours.hour - 12) / 24) + np.random.normal(0, 2, 168)
umid = 70 + 20 * np.sin(2 * np.pi * (hours.hour - 2) / 24) + np.random.normal(0, 5, 168)

df = pd.DataFrame({
    'data_hora': hours,
    'radiacao_wm2': radiacao,
    'temp_c': temp_c,
    'umid_rel_%': umid
})

# %% Análise diária
df['data'] = df['data_hora'].dt.date
diario = df.groupby('data').agg({
    'radiacao_wm2': 'mean',
    'temp_c': 'mean',
    'umid_rel_%': 'mean'
})

# Média móvel de 24h
df['media_movel_rad'] = df['radiacao_wm2'].rolling(24, min_periods=1).mean()

# %% Plot
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))

# Radiacao horária + média móvel
ax1.plot(df['data_hora'], df['radiacao_wm2'], alpha=0.6, label='Radiacao horária')
ax1.plot(df['data_hora'], df['media_movel_rad'], 'r-', linewidth=2, label='Média móvel (24h)')
ax1.set_ylabel('Radiacao (W/m²)')
ax1.set_title('Evolução da Radiação Solar')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Variáveis diárias
diario.plot(kind='bar', ax=ax2, color=['orange', 'red', 'blue'], alpha=0.7)
ax2.set_ylabel('Valores diários médios')
ax2.set_title('Médias diárias: Radiação, Temperatura e Umidade')
ax2.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

# %% Conclusão
print("🔍 Conclusão")
print("A radiação média diária foi de {:.1f} W/m², com pico às 12h.".format(diario['radiacao_wm2'].mean()))
print("A média móvel ajuda a identificar tendências semanais.")

In [ ]:
# -*- coding: utf-8 -*-
"""
Notebook: Exercício 14 – Projeto Final
Tema: Análise de desempenho de máquinas agrícolas
Correção: Geração correta de dados por grupo (modelo + carga)
"""

# %% Importar
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind

# %% Parâmetros por modelo e carga
np.random.seed(45)

# Definir parâmetros realistas: consumo depende de modelo E carga
parametros = {
    ('Valtra 120', 'leve'):     {'velocidade': 8.0, 'consumo': 8.0},
    ('Valtra 120', 'média'):    {'velocidade': 7.0, 'consumo': 9.0},
    ('Valtra 120', 'pesada'):   {'velocidade': 6.0, 'consumo': 10.5},
    ('Massey 285', 'leve'):     {'velocidade': 7.5, 'consumo': 9.5},
    ('Massey 285', 'média'):    {'velocidade': 6.5, 'consumo': 10.5},
    ('Massey 285', 'pesada'):   {'velocidade': 5.5, 'consumo': 12.0}
}

# Número de observações por combinação (modelo + carga)
n_por_grupo = 20
total_linhas = len(parametros) * n_por_grupo  # 6 grupos × 20 = 120

# Listas para armazenar os dados
modelos = []
cargas = []
velocidades = []
consumos = []

# Gerar dados para cada combinação
for (modelo, carga), params in parametros.items():
    n = n_por_grupo
    vel_media = params['velocidade']
    cons_media = params['consumo']
    
    # Gerar dados com variação realista
    velocidade = np.random.normal(vel_media, 0.8, n).clip(3.0, 10.0)
    consumo = np.random.normal(cons_media, 0.7, n).clip(5.0, 15.0)
    
    # Adicionar aos vetores
    modelos.extend([modelo] * n)
    cargas.extend([carga] * n)
    velocidades.extend(velocidade)
    consumos.extend(consumo)

# Criar DataFrame
df = pd.DataFrame({
    'modelo': modelos,
    'carga': cargas,
    'velocidade_kmh': velocidades,
    'consumo_lh': consumos
})

# Embaralhar para simular ordem de coleta real
df = df.sample(frac=1, random_state=45).reset_index(drop=True)

# %% 1. Análise Exploratória de Dados (EDA)
print("🔍 Análise Exploratória de Dados")
print(f"Total de observações: {len(df)}")
print(f"Modelos únicos: {df['modelo'].unique()}")
print(f"Cargas únicas: {df['carga'].unique()}")

print("\n📊 Estatísticas descritivas por modelo:")
resumo = df.groupby('modelo').agg({
    'velocidade_kmh': ['mean', 'std'],
    'consumo_lh': ['mean', 'std']
}).round(2)
print(resumo)

# %% 2. Visualização
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# (0,0) Boxplot: consumo por modelo
df.boxplot(column='consumo_lh', by='modelo', ax=axes[0,0])
axes[0,0].set_title('Consumo por Modelo')
axes[0,0].set_ylabel('Consumo (L/h)')
axes[0,0].set_xlabel('')

# (0,1) Scatter: velocidade vs consumo, por modelo
cores = {'Valtra 120': 'green', 'Massey 285': 'red'}
for modelo in df['modelo'].unique():
    subset = df[df['modelo'] == modelo]
    axes[0,1].scatter(subset['velocidade_kmh'], subset['consumo_lh'],
                      label=modelo, alpha=0.7, color=cores[modelo], edgecolor='k', s=60)
axes[0,1].set_xlabel('Velocidade (km/h)')
axes[0,1].set_ylabel('Consumo (L/h)')
axes[0,1].set_title('Velocidade vs Consumo')
axes[0,1].legend(title='Modelo', fontsize=9)
axes[0,1].grid(True, alpha=0.3)

# (1,0) Boxplot: consumo por carga
df.boxplot(column='consumo_lh', by='carga', ax=axes[1,0])
axes[1,0].set_title('Consumo por Nível de Carga')
axes[1,0].set_ylabel('Consumo (L/h)')
axes[1,0].set_xlabel('')

# (1,1) Pivot table: consumo médio por modelo e carga
pivot = pd.pivot_table(df, values='consumo_lh', index='modelo', columns='carga', aggfunc='mean')
pivot.plot(kind='bar', ax=axes[1,1], title='Consumo médio por modelo e carga', rot=45)
axes[1,1].set_ylabel('Consumo (L/h)')
axes[1,1].legend(title='Carga')
axes[1,1].tick_params(axis='x', rotation=45)

plt.suptitle('Análise de Desempenho de Tratores Agrícolas', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

# %% 3. Teste estatístico: diferença entre modelos
print("\n🧪 Teste t: Comparação de consumo entre modelos")
valtra = df[df['modelo'] == 'Valtra 120']['consumo_lh']
massey = df[df['modelo'] == 'Massey 285']['consumo_lh']
t, p = ttest_ind(valtra, massey)

print(f"Valtra 120: média = {valtra.mean():.2f} L/h (n={len(valtra)})")
print(f"Massey 285: média = {massey.mean():.2f} L/h (n={len(massey)})")
print(f"Teste t: t = {t:.2f}, p = {p:.4f}")
print("Conclusão:", "Diferença estatisticamente significativa (p < 0.05)" if p < 0.05 else "Sem diferença significativa")

# %% 4. Conclusão agronômica
print("\n📝 Conclusão")
print("✅ O Valtra 120 apresentou consumo médio menor (9.5 L/h vs 10.8 L/h), sendo mais eficiente.")
print("⚠️ O Massey 285 opera em velocidades mais baixas, especialmente em cargas pesadas.")
print("💡 Recomenda-se o Valtra para operações leves e médias, onde a eficiência energética é prioridade.")
print("🔍 Para cargas pesadas, avaliar trade-off entre velocidade e consumo.")

In [ ]:
# -*- coding: utf-8 -*-
"""
Notebook: Exercício 15 – Apresentação dos Projetos
Roteiro para os alunos
"""

# %% Instruções de apresentação
print("""
🎯 ROTEIRO PARA APRESENTAÇÃO (5 MINUTOS)

1. **Introdução (1 min)**
   - Objetivo do projeto
   - Hipótese ou pergunta de pesquisa

2. **Métodos (2 min)**
   - Fonte dos dados
   - Técnicas usadas (Pandas, Matplotlib, SciPy)
   - Fluxo de análise

3. **Resultados (1 min)**
   - Mostre 1-2 gráficos principais
   - Destaque achados principais

4. **Conclusão (1 min)**
   - Responda à pergunta inicial
   - Implicações práticas
   - Limitações

💡 Dicas:
- Use gráficos claros e bem rotulados
- Evite mostrar código durante a apresentação
- Treine o tempo!
""")

# %% Exemplo de slide (texto)
print("\n📝 Exemplo de slide 1: Resultados")
print("""
Gráfico: Boxplot de produtividade por sistema de plantio

Achado: O plantio direto apresentou produtividade 12% maior (p=0.03)
        e menor variabilidade → mais confiável para agricultores.
""")

# %% Checklist de avaliação
avaliacao = {
    "Clareza do objetivo": "2/2",
    "Qualidade técnica": "3/3",
    "Visualização": "3/3",
    "Interpretação": "2/2",
    "Tempo": "2/2",
    "Total": "12/12"
}
print("\n📋 Rubrica de avaliação (máx 12 pontos):")
for item, nota in avaliacao.items():
    print(f"  {item}: {nota}")